## MUSCL Schemes

In this notebook, we will explore high-order finite volume via MUSCL reconstruction.

To run each of the following cells, use the keyboard shortcut **SHIFT** + **ENTER**, press the button ``Run`` in the toolbar or find the option ``Cell > Run Cells`` from the menu bar. For more shortcuts, see ``Help > Keyboard Shortcuts``.

To get started, import the required Python modules by running the cell below.

In [ ]:
# Configuration for visualizing the plots
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

# Required modules
import numpy as np
import matplotlib.pyplot as plt

Import figure style and custom functions
import nbtools as nb

We wil implement finite-volume methods based on MUSCL reconstruction to solve 

\begin{align}
	\frac{\partial u}{\partial t} + \frac{\partial u}{\partial x} = 0
\end{align}
on a grid $x\in[0,2]$ with periodic boundary conditions. 

Following the example in the textbook, we implement ``residual_muscl`` to compute the residual using the second-order upwind-biased method. For the advection scheme, we use a Riemann solver of the form $F_(u_L,u_R) = F(u_L)$. 

**Note**: Only ``dl`` is implemented due to the upwind Riemann solver. Later, you will need to implement the right slope ``dr``.

In [ ]:
def get_slope(u0, u1, u2, r):
    # Linear reconstruction
    return 0.5*(1 + r)*(u1 - u0) + 0.5*(1 - r)*(u2 - u1)

def residual(u, a, dx):
    res = 0.0*u
    r = 1.0
    for i in range(n):
        # Interface i+1/2
        if n == n - 1:
            # Enforce periodic BC at x=L
            dl = get_slope(u[i - 1], u[i], u[0], r)
        else:
            dl = get_slope(u[i - 1], u[i], u[i + 1], r)
            
        # Compute u_i+1/2,L
        ul = 
        
        # Compute f_i+1/2 using the upwind Riemann solver
        fupw_R = 
        
        # Interface i-i/2
        dl = get_slope(u[i - 2], u[i - 1], u[i], r) 
        
        # Compute u_i-1/2,L
        ul = 
        
        # Computr f_i-1/2 using the upwind Rieman solver
        fupw_L = 
        
        res[i] = -(fupw_R - fupw_L)/dx
    return res

To advance the solution in time, we provide the second-order midpoint method from the Time-Stepping Methods notebook.

In [ ]:
def advance_solution(ut, a, dx, dt, tf):
    t = 0
    while t < tf:
        r = residual(ut, a, dx)
        um = ut + 0.5*dt*r
        rm = residual(um, a, dx)
        uf = ut + 1.0*dt*rm
        ut = 1.0*uf
        t += dt 
    return ut

The initial solution profile is given by
\begin{align}
	u(x, 0) &= 
	\begin{cases}
		e^{-20(x-0.5)^2} & \text{if } x < 1.2, \\ 
		1 & \text{if } 1.2 < x < 1.5,	 \\
		0 & \text{otherwise}.
	\end{cases}
\end{align}
Complete the following cell to set the initial parameters of the problem. Consider
- ``a = 1.0``
- ``n = 300``
- ``L = 2.0``
- ``dt = 2e-3``
- ``tf = 2.0``


In [ ]:
a = 
n = 
L = 
dt = 
tf = 

# Calculate mesh spacing and grid points
dx = L/n 
x = np.arange(0, L, dx)

# Initialize the solution
u0 = np.zeros(n)
for i, xval in enumerate(x):
    if xval < 1.2:
        u0[i] =
    if 1.2 < xval < 1.5:
        u0[i] = 
    else:
        u0[i] = 

Advance the solution using the time-stepping function provided above

In [ ]:
u_adv = advance_solution(u0, a, dx, dt, tf)

Compute the total variation (TV) of the initial and the final solution. Is this method TVD?

Complete the TV calculation of the final solution.

In [ ]:
tv0 = np.sum(np.abs(u0[1:]-u[:-1]))
tvf = 

if tvf <= tv0:
    print('Scheme is TVD')
else:
    print('Scheme is not TVD')

Plot your result for each implemented limiter

In [ ]:
plt.figure(0)
plt.plot(x, u_adv)

To preserve the monotonicity of the scheme, we need to define limiter functions. Complete the ``van_leer`` and ``superbee`` limiters using the formulations shown in the textbook. Note the use of a tolerance variable ``1e-20`` to avoid divisions by zero.

In [ ]:
def minmod(a, b):
    r = a/(b + 1e-20)
    return max([0, min([1, r])])

def van_leer(a, b):
    # Complete limiter code
    
def superbee(a, b):
    # Complete limiter code

Modify the definition of the ``residual`` function (or rewrite it below) to include the limiters following the theory in the textbook. To facilitate the implementation, use the variable ``limiter(a,b)`` to obtain the values of the limiter. 

**Note**: No modification of the arguments in ``residual`` is necessary.

For each of the limiter functions, generate a plot and compute the total variation

In [ ]:
plt.figure(1)
labels = ['minmod', 'van Leer', 'superbee']
for i, limiter in zip([minmod, van_leer, superbee]):
    u_adv = advance_solution(u0, a, dx, dt, tf)
    plt.plot(x, u_adv, label=labels[i])
plt.legend()

## Activities
- Modify the definition of ``get_slope`` to obtain a quadratic approximation of the slope at the boundaries. Repeat the exercises in the notebook using the new order. 
- What are the advantages and disadvantages of increasing the order of the scheme?
- Modify the definition of ``residual`` to use a Riemann solver of the form $F(u_L, u_R)$. To achieve this, implement a function named ``riemann_solver`` and compute the ``dr`` slopes at each interface.
- Rewrite ``riemann_solver`` using a Roe solver, to be used with the Burgers equation.